In [1]:
from tables import *

In [2]:
# Apaga todas as tabelas do banco
Base.metadata.drop_all(bind = engine)
# cria todas novamente
Base.metadata.create_all(engine)

In [3]:
# Adicionando atores
global genre1, genre2, movie1, movie2
with Session() as session:
    actor1 = actor(name="Robert Downey Jr.")
    actor2 = actor(name="Chris Evans")
    session.add(actor1)
    session.add(actor2)

    # Adicionando gêneros
    genre1 = movieGeneres(name="Action")
    genre2 = movieGeneres(name="Adventure")
    session.add(genre1)
    session.add(genre2)

     # Commitando as mudanças
    session.commit()    

    # Adicionando filmes
    movie1 = movie(title="Iron Man", studio="Marvel Studios", genere_id = genre1.id)
    movie2 = movie(title="Captain America", studio="Marvel Studios", genere_id = genre2.id)
    session.add(movie1)
    session.add(movie2)

    # Associando atores a filmes
    movie_actor1 = movieActors(actor_id=actor1.id, movie_id=movie1.id)  # Robert Downey Jr. em Iron Man
    movie_actor2 = movieActors(actor_id=actor2.id, movie_id=movie2.id)
    session.add(movie_actor1)
    session.add(movie_actor2)

    director1 = director(name="Jon Favreau")
    director2 = director(name="Joe Johnston")
    session.add(director1)
    session.add(director2)

    movie_director1 = movieDirectors(movie_id=movie1.id, directors_id=movie1.id)  # Jon Favreau dirigiu Iron Man
    movie_director2 = movieDirectors(movie_id=movie2.id, directors_id=movie2.id)  # Joe Johnston dirigiu Captain America
    session.add(movie_director1)
    session.add(movie_director2)

    
    # Adicionando clientes
    client1 = client(name="Alice")
    client2 = client(name="Bob")
    
    session.add(client1)
    session.add(client2)  
    session.commit()
    
    review1 = sessionReview(client_id=client1.id, rating=5, review="Amazing movie!")
    review2 = sessionReview(client_id=client2.id, rating=4, review="Great movie!")

    session.add(review1)
    session.add(review2)
    session.commit()    
    
    session1 = MovieSessions(movie_id=movie1.id, sessionReview_id = review1.id, session_date=datetime(2024, 8, 1, 18, 0))
    session2 = MovieSessions(movie_id=movie2.id, sessionReview_id = review2.id, session_date=datetime(2024, 7, 2, 18, 0))

    session.add(session1)
    session.add(session2)
    session.commit()    

    # Associando sessões a clientes
    session_client1 = sessionClient(session_id=session1.id, movie_id=movie1.id)
    session_client2 = sessionClient(session_id=session2.id, movie_id=movie2.id)
    session.add(session_client1)
    session.add(session_client2)


    session.commit()    

In [4]:
sql_query = '''
SELECT Movie.title, SessionReview.rating
FROM MovieSessions
JOIN Movie ON MovieSessions.movie_id = Movie.id
JOIN SessionReview ON MovieSessions.sessionReview_id = SessionReview.id
ORDER BY SessionReview.rating DESC;
''' 
df = pd.read_sql(sql_query, engine)

df.head()

,title,rating
0,Iron Man,5
1,Captain America,4


In [5]:
sql_query = '''
SELECT Movie.title, SessionReview.rating
FROM MovieSessions
JOIN Movie ON MovieSessions.movie_id = Movie.id
JOIN SessionReview ON MovieSessions.sessionReview_id = SessionReview.id
ORDER BY Movie.title;
'''
df = pd.read_sql(sql_query, engine)

df.head()

,title,rating
0,Captain America,4
1,Iron Man,5


In [6]:
sql_query = '''
SELECT movie.title, sessionReview.review, MovieSessions.session_date
FROM MovieSessions
JOIN movie ON movie.id = MovieSessions.movie_id
JOIN sessionReview ON sessionReview.id = MovieSessions.sessionReview_id
ORDER BY MovieSessions.session_date DESC;
''' 
df = pd.read_sql(sql_query, engine)

df.head()

,title,review,session_date
0,Iron Man,Amazing movie!,2024-08-01 18:00:00.000000
1,Captain America,Great movie!,2024-07-02 18:00:00.000000


In [7]:
from sqlalchemy import create_engine, Column, Integer, String, MetaData, ForeignKey, DateTime, text

sql_query = '''
SELECT movie.title, sessionReview.review, MovieSessions.session_date
FROM MovieSessions
JOIN movie ON movie.id = MovieSessions.movie_id
JOIN sessionReview ON sessionReview.id = MovieSessions.sessionReview_id
Where movie.title = "{title}"
ORDER BY MovieSessions.session_date DESC;
''' 
df = pd.read_sql(text(sql_query.format(title = 'Iron Man')), engine)

df.head()

,title,review,session_date
0,Iron Man,Amazing movie!,2024-08-01 18:00:00.000000
